# Introducción

Los maps nos permiten transformar datos en un DataFrame o Serie, un valor a la vez para una columna completa. Sin embargo, a menudo queremos agrupar nuestros datos y luego hacer algo específico para el grupo en el que se encuentran los datos.

Como aprenderá, hacemos esto con la operación groupby(). También cubriremos algunos temas adicionales, como formas más complejas de indexar sus DataFrames, además de cómo ordenar sus datos.

**Para iniciar el ejercicio de este tema, haga clic [aquí](https://www.kaggle.com/kernels/fork/598715).**

# Groupwise analysis

Una función que hemos estado usando mucho hasta ahora es la función value_counts(). Podemos replicar lo que hace value_counts() haciendo lo siguiente:

In [20]:

import pandas as pd
reviews = pd.read_csv("../Datas/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)
reviews

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,90,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit


In [6]:
reviews.groupby('points').points.count()

points
80     397
81     692
      ... 
99      33
100     19
Name: points, Length: 21, dtype: int64

`groupby()` creó un grupo de reseñas que asignaban los mismos valores de puntos a los vinos determinados. Luego, para cada uno de estos grupos, tomamos la columna de `points()` y contamos cuántas veces apareció. `value_counts() `es sólo un atajo a esta operación `groupby()`

Podemos usar cualquiera de las funciones de resumen que hemos usado antes con estos datos. Por ejemplo, para conseguir el vino más barato en cada categoría de valor en puntos, podemos hacer lo siguiente:

In [7]:
reviews.groupby('points').price.min()

points
80      5.0
81      5.0
       ... 
99     44.0
100    80.0
Name: price, Length: 21, dtype: float64

Puede pensar en cada grupo que generamos como una porción de nuestro DataFrame que contiene solo datos con valores que coinciden. Podemos acceder a este DataFrame directamente mediante el método `apply()`, y luego podemos manipular los datos de la forma que consideremos adecuada. Por ejemplo, aquí hay una forma de seleccionar el nombre del primer vino revisado de cada bodega en el conjunto de datos:

In [8]:
reviews.groupby('winery').apply(lambda df: df.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

Para un control aún más detallado, también puede agrupar por más de una columna. Por ejemplo, así es como elegiríamos el mejor vino por país y provincia:

In [11]:
reviews.groupby(['country', 'province']).apply(lambda df: df.loc[df.points.idxmax()])

country  \
country   province                      
Argentina Mendoza Province  Argentina   
          Other             Argentina   
...                               ...   
Uruguay   San Jose            Uruguay   
          Uruguay             Uruguay   

                                                                  description  \
country   province                                                              
Argentina Mendoza Province  If the color doesn't tell the full story, the ...   
          Other             Take note, this could be the best wine Colomé ...   
...                                                                       ...   
Uruguay   San Jose          Baked, sweet, heavy aromas turn earthy with ti...   
          Uruguay           Cherry and berry aromas are ripe, healthy and ...   

                                          designation  points  price  \
country   province                                                     
Argentina Mendoza Province           Nicasia Vineyard      97  120.0   
          Other                               Reserva      95   90.0   
...                                               ...     ...    ...   
Uruguay   San Jose           El Preciado Gran Reserva      87   50.0   
          Uruguay           Blend 002 Limited Edition      91   22.0   

                                    province region_1 region_2  \
country   province                                               
Argentina Mendoza Province  Mendoza Province  Mendoza      NaN   
          Other                        Other    Salta      NaN   
...                                      ...      ...      ...   
Uruguay   San Jose                  San Jose      NaN      NaN   
          Uruguay                    Uruguay      NaN      NaN   

                                  taster_name taster_twitter_handle  \
country   province                                                    
Argentina Mendoza Province  Michael Schachner           @wineschach   
          Other             Michael Schachner           @wineschach   
...                                       ...                   ...   
Uruguay   San Jose          Michael Schachner           @wineschach   
          Uruguay           Michael Schachner           @wineschach   

                                                                        title  \
country   province                                                              
Argentina Mendoza Province  Bodega Catena Zapata 2006 Nicasia Vineyard Mal...   
          Other                            Colomé 2010 Reserva Malbec (Salta)   
...                                                                       ...   
Uruguay   San Jose          Castillo Viejo 2005 El Preciado Gran Reserva R...   
          Uruguay           Narbona NV Blend 002 Limited Edition Tannat-Ca...   

                                          variety                winery  
country   province                                                       
Argentina Mendoza Province                 Malbec  Bodega Catena Zapata  
          Other                            Malbec                Colomé  
...                                           ...                   ...  
Uruguay   San Jose                      Red Blend        Castillo Viejo  
          Uruguay           Tannat-Cabernet Franc               Narbona  

[425 rows x 13 columns]

Otro `groupby()` método que vale la pena mencionar es `agg()`, lo que le permite ejecutar un montón de funciones diferentes en su DataFrame simultáneamente. Por ejemplo, podemos generar un resumen estadístico simple del conjunto de datos de la siguiente manera:

In [14]:
reviews.groupby(['country']).price.agg([len, min, max])

,len,min,max
country,,,
Argentina,3800,4.0,230.0
Armenia,2,14.0,15.0
...,...,...,...
Ukraine,14,6.0,13.0
Uruguay,109,10.0,130.0


El uso eficaz de `groupby()` le permitirá hacer muchas cosas realmente poderosas con su conjunto de datos.

# Multi-indexes

En todos los ejemplos que hemos visto hasta ahora hemos estado trabajando con objetos DataFrame o Series con un índice de etiqueta única. `groupby()` es ligeramente diferente en el hecho de que, dependiendo de la operación que ejecutemos, a veces dará como resultado lo que se llama un índice múltiple (multi-index).

Un índice múltiple se diferencia de un índice normal en que tiene múltiples niveles. Por ejemplo:

In [15]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed

len
country   province              
Argentina Mendoza Province  3264
          Other              536
...                          ...
Uruguay   San Jose             3
          Uruguay             24

[425 rows x 1 columns]

In [16]:
mi = countries_reviewed.index
type(mi)

pandas.core.indexes.multi.MultiIndex

tienen varios métodos para manejar su estructura escalonada que están ausentes en los índices de un solo nivel. También requieren dos niveles de etiquetas para recuperar un valor. Tratar con resultados de múltiples índices es un problema común para los usuarios nuevos en pandas.

Los casos de uso para un índice múltiple se detallan junto con las instrucciones sobre su uso en la [MultiIndex / Advanced Selection](https://pandas.pydata.org/pandas-docs/stable/advanced.html) sección de la documentación de pandas.

Sin embargo, en general, el método de índices múltiples que utilizará con más frecuencia es el de volver a convertir a un índice normal. el metodo `reset_index()`:

In [17]:
countries_reviewed.reset_index()

,country,province,len
0,Argentina,Mendoza Province,3264
1,Argentina,Other,536
...,...,...,...
423,Uruguay,San Jose,3
424,Uruguay,Uruguay,24


# Sorting

Mirando de nuevo a `countries_reviewed` Podemos ver que la agrupación devuelve datos en orden de índice, no en orden de valor. Es decir, al generar el resultado de un `groupby()`, el orden de las filas depende de los valores del índice, no de los datos.

Para obtener los datos en el orden en que los queremos, podemos ordenarlos nosotros mismos. El método `sort_values()` es útil para esto.

In [18]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by='len')

,country,province,len
179,Greece,Muscat of Kefallonian,1
192,Greece,Sterea Ellada,1
...,...,...,...
415,US,Washington,8639
392,US,California,36247


`sort_values()` El orden predeterminado es ascendente, donde los valores más bajos van primero. Sin embargo, la mayoría de las veces queremos una clasificación descendente, donde los números más altos van primero. Eso va así:

In [19]:
countries_reviewed.sort_values(by='len', ascending=False)

,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


Para ordenar por valores de índice, utilice el método complementario `sort_index()`. Este método tiene los mismos argumentos y orden predeterminado:

In [ ]:
countries_reviewed.sort_index()

Por último, sepa que puede ordenar por más de una columna a la vez:

In [ ]:
countries_reviewed.sort_values(by=['country', 'len'])

# Your turn

If you haven't started the exercise, you can **[get started here](https://www.kaggle.com/kernels/fork/598715)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*